daily_run estimates strikes and number of contracts for a put selling strategy on equity indices such as S&P 500. 

Algorithm is drawn from  JUREK, J. W. and STAFFORD, E. (2015), The Cost of Capital for Alternative Investments. The Journal of Finance, 70: 2185-2226. doi:10.1111/jofi.12269- 

Requires an open session of either IB Gateway or TWS from Interactive Brokers

In [1]:
from option_daily_prod import OptionMarket, SpxOptionAsset, RSL2OptionAsset
import numpy as np
import pandas as pd
import nest_asyncio
from time import time

before = time()
nest_asyncio.apply()

spx_trades = OptionMarket(SpxOptionAsset())
rsl_trades = OptionMarket(RSL2OptionAsset())

In [23]:
z_score = np.array([-1, -1.5, -2])
leverage = np.array(range(1, 4, 1))
num_expiries = 3
illiquid_equity = 500000
after = time()
spx = True
if spx:
    trd_choice = spx_trades.form_trade_choice(z_score, num_expiries, 'P')
else:
    trd_choice = rsl_trades.form_trade_choice(z_score, num_expiries, 'P')
    
last_price = trd_choice.spot
after_2 = time()
print('elapsed:', np.round(after - before, 1))
print('elapsed:', np.round(after_2 - before, 1))
print(last_price)
print(trd_choice.sigma)
print(trd_choice.account_value)

elapsed: 3035.5
elapsed: 3047.7
2724.66
20.42
[AccountValue(account='U2463043', tag='NetLiquidationByCurrency', value='2762750.8514', currency='BASE', modelCode='')]


In [24]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction:row;
}
"""
HTML('<style>{}</style>'.format(CSS))


In [25]:
_=[display(df) for df in [trd_choice.strike_grid(), trd_choice.premium_grid(), trd_choice.prices_grid()]]

,-1.0,-1.5,-2.0
2018.12.21,2590,2525,2480
2019.01.18,2530,2435,2365
2019.02.15,2465,2345,2255


,-1.0,-1.5,-2.0
2018.12.21,15.05,8.20,5.50
2019.01.18,21.35,12.05,8.05
2019.02.15,25.85,14.75,9.80


,-1.0,-1.5,-2.0
2018.12.21,14.9/15.2,8.1/8.3,5.4/5.6
2019.01.18,21.1/21.6,11.9/12.2,7.9/8.2
2019.02.15,25.5/26.2,14.5/15.0,9.6/10.0


In [16]:

prct_decline = trd_choice.pct_otm_grid(last_price)
format_dict = {x: '{:.2%}' for x in prct_decline.to_dict()}
display(prct_decline.style.format(format_dict))

,-1.0,-1.5,-2.0
2018.12.21,-4.88%,-7.08%,-8.91%
2019.01.18,-7.08%,-10.55%,-12.93%
2019.02.15,-9.45%,-13.84%,-16.95%


In [17]:
capital_at_risk = float(trd_choice.account_value[0].value) + illiquid_equity
premium_list = []
premium_list =  [(trd_choice.premium_grid() * contractTable.round() * 100) \
                 for contractTable in trd_choice.option_lots(leverage, capital_at_risk)]

_ = [display(df.style.format('${:,.0f}')) for df in premium_list]  

,-1.0,-1.5,-2.0
2018.12.21,"$19,435","$11,115","$7,215"
2019.01.18,"$27,430","$15,470","$11,410"
2019.02.15,"$33,020","$20,230","$14,700"


,-1.0,-1.5,-2.0
2018.12.21,"$37,375","$22,230","$14,430"
2019.01.18,"$54,860","$32,130","$22,820"
2019.02.15,"$68,580","$40,460","$28,420"


,-1.0,-1.5,-2.0
2018.12.21,"$56,810","$33,345","$21,645"
2019.01.18,"$82,290","$47,600","$33,415"
2019.02.15,"$101,600","$60,690","$43,120"


In [18]:
premium_list_pct = []
premium_list_pct = [(trd_choice.premium_grid() * lev) / last_price for lev in leverage]
_ = [display(df.style.format('{:.2%}')) for df in premium_list_pct]  

,-1.0,-1.5,-2.0
2018.12.21,0.55%,0.31%,0.20%
2019.01.18,0.77%,0.44%,0.30%
2019.02.15,0.93%,0.53%,0.36%


,-1.0,-1.5,-2.0
2018.12.21,1.09%,0.63%,0.41%
2019.01.18,1.54%,0.87%,0.60%
2019.02.15,1.86%,1.06%,0.72%


,-1.0,-1.5,-2.0
2018.12.21,1.64%,0.94%,0.61%
2019.01.18,2.32%,1.31%,0.89%
2019.02.15,2.79%,1.59%,1.08%


In [19]:

_ = [display(contractTable.round().style.format('{:.0f}')) for \
     contractTable in trd_choice.option_lots(leverage, capital_at_risk)]
    

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,13,13,13
2019.01.18,13,13,14
2019.02.15,13,14,15


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,25,26,26
2019.01.18,26,27,28
2019.02.15,27,28,29


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,38,39,39
2019.01.18,39,40,41
2019.02.15,40,42,44


In [20]:
margin_dlr = [trd_choice.margin(last_price) * df for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in margin_dlr]  

,-1.0,-1.5,-2.0
2018.12.21,"$556,678","$470,358","$401,458"
2019.01.18,"$486,673","$351,213","$344,610"
2019.02.15,"$407,763","$349,930","$355,200"


,-1.0,-1.5,-2.0
2018.12.21,"$1,070,535","$940,716","$802,916"
2019.01.18,"$973,346","$729,443","$689,220"
2019.02.15,"$846,893","$699,860","$686,720"


,-1.0,-1.5,-2.0
2018.12.21,"$1,627,213","$1,411,075","$1,204,375"
2019.01.18,"$1,460,020","$1,080,656","$1,009,215"
2019.02.15,"$1,254,656","$1,049,790","$1,041,920"


In [10]:
marge = trd_choice.margin(trd_choice.spot)
lots = trd_choice.option_lots([2], capital_at_risk)
margin = lots[0] * marge
display(margin.style.format('${:,.0f}'))

,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$1,080,395","$951,361","$827,471"
2019.01.18,"$983,861","$755,347","$692,020"
2019.02.15,"$873,337","$702,940","$691,940"


In [11]:
notional_dlr=[df * trd_choice.strike_grid() * 100 for df in trd_choice.option_lots(leverage, capital_at_risk)]
_ = [display(df.style.format('${:,.0f}')) for df in notional_dlr]  

,-1.0,-1.5,-2.0
Lev 1,,,
2018.12.21,"$3,386,500","$3,308,500","$3,250,000"
2019.01.18,"$3,308,500","$3,191,500","$3,346,000"
2019.02.15,"$3,230,500","$3,311,000","$3,199,000"


,-1.0,-1.5,-2.0
Lev 2,,,
2018.12.21,"$6,512,500","$6,617,000","$6,500,000"
2019.01.18,"$6,617,000","$6,628,500","$6,692,000"
2019.02.15,"$6,709,500","$6,622,000","$6,626,500"


,-1.0,-1.5,-2.0
Lev 3,,,
2018.12.21,"$9,899,000","$9,925,500","$9,750,000"
2019.01.18,"$9,925,500","$9,820,000","$9,799,000"
2019.02.15,"$9,940,000","$9,933,000","$9,825,500"


In [12]:
capital_at_risk

3265240.1952